In [82]:
import numpy as np

In [83]:
def DMD(X, Y, tol=1e-12):
    #X - Nxm, Y - Nxm
    m=X.shape[1]; N=X.shape[0]
    U, Sigma, V = np.linalg.svd(X,full_matrices=False) #svd vraca V*
    V=np.conjugate(V.T)
    #print(Sigma)
    k=m
    for i in range(1,min(N,m)):
        if(Sigma[i]<=Sigma[0]*tol):
            k=i
            break
    U = U[:, :k]; V=V[:, :k]; Sigma = np.array(Sigma[:k])
    S_k = ((np.conjugate(U.T)@Y)@V)/Sigma
    Lambda, W = np.linalg.eig(S_k)
    Z = U@W
    return Z, Lambda


In [84]:
def DMD_exact(X, Y, tol=1e-12):
    """X - Nxm, Y - Nxm; !!Exact DMD ne vraća normirane Z[:,i]"""
    m=X.shape[1]; N=X.shape[0]
    U, Sigma, V = np.linalg.svd(X,full_matrices=False) #svd vraca V*
    V=np.conjugate(V.T)
    #print(Sigma)
    k=m
    for i in range(1,min(N,m)):
        if(Sigma[i]<=Sigma[0]*tol):
            k=i
            break
    U = U[:, :k]; V=V[:, :k]; Sigma = np.array(Sigma[:k])
    S_k = ((np.conjugate(U.T)@Y)@V)/Sigma
    Lambda, W = np.linalg.eig(S_k)
    Z = (Y@V/Sigma)@W/Lambda
    return Z, Lambda

In [95]:
N=300
m=10
x=np.random.rand(N)
A=np.random.rand(N,N)
A = A/np.linalg.norm(A, 2)

In [96]:
X=np.empty((N,m+1))
X[:,0]=x
for i in range(m):
    X[:,i+1]=A@X[:,i]
X=X+np.random.rand(N,m+1)/N

In [97]:
Z, L = DMD(X[:,:m], X[:, 1:], N*np.finfo(float).resolution)
#Z ima stupčanu normu otprilike 1
for i in range(Z.shape[1]):
    print(np.linalg.norm(Z[:,i]))
L

1.0000000000000002
0.9999999999999999
0.9999999999999999
1.0000000000000002
1.0000000000000002
1.0000000000000002
1.0000000000000002
1.0
1.0
1.0


array([ 0.99906284+0.j        ,  0.57634157+0.66141524j,
        0.57634157-0.66141524j, -0.11012703+0.8096703j ,
       -0.11012703-0.8096703j ,  0.02545672+0.j        ,
       -0.01056118+0.j        , -0.73300498+0.j        ,
       -0.59640317+0.49457968j, -0.59640317-0.49457968j])

In [98]:
Ze, Le = DMD_exact(X[:,:m], X[:, 1:], N*np.finfo(float).resolution)
Le
#isto kao L?

array([ 0.99906284+0.j        ,  0.57634157+0.66141524j,
        0.57634157-0.66141524j, -0.11012703+0.8096703j ,
       -0.11012703-0.8096703j ,  0.02545672+0.j        ,
       -0.01056118+0.j        , -0.73300498+0.j        ,
       -0.59640317+0.49457968j, -0.59640317-0.49457968j])

In [99]:
np.linalg.eigvals(A)[:10]

array([ 0.99902998+0.j        , -0.03226574+0.01282388j,
       -0.03226574-0.01282388j, -0.03409987+0.j        ,
       -0.01769994+0.02909243j, -0.01769994-0.02909243j,
        0.00302156+0.03364694j,  0.00302156-0.03364694j,
       -0.01684775+0.02782427j, -0.01684775-0.02782427j])

In [100]:
#!pip install pydmd

In [101]:
import pydmd
dmd1 = pydmd.DMD(svd_rank=10)
dmd1.fit(X[:,:m], X[:,1:])
dmd1.eigs

array([ 0.99906284+0.j        ,  0.57634157+0.66141524j,
        0.57634157-0.66141524j, -0.11012703+0.8096703j ,
       -0.11012703-0.8096703j ,  0.02545672+0.j        ,
       -0.01056118+0.j        , -0.73300498+0.j        ,
       -0.59640317+0.49457968j, -0.59640317-0.49457968j])

In [102]:
for i in range(dmd1.eigs.shape[0]):
    print(np.linalg.norm(dmd1.reconstructed_data[:,i]-X[:,i]))

9.033802718357805e-15
2.106177684303512e-14
3.66742698172252e-14
5.699502009271928e-14
8.139224648981389e-14
1.0331403214836834e-13
1.1887923027270711e-13
1.4666684958597648e-13
1.6251941276142768e-13
1.8350713198303222e-13


In [103]:
dmd1.reconstructed_data.shape

(300, 10)

In [104]:
#kako procijeniti s_i?
for i in range(m):
    s=np.zeros((Z.shape[0]))
    for j in range(Z.shape[1]):
        s=s+Z[:,j]*(L[j]**(i))
    print("norma razlike:",np.linalg.norm(s-X[:,i]))
    #print(s,X[:,i], sep="\n")

norma razlike: 11.504003460297413
norma razlike: 9.865137420348548
norma razlike: 9.80429018238895
norma razlike: 9.883126730935865
norma razlike: 9.946794403575232
norma razlike: 9.939925870504666
norma razlike: 9.775210408701628
norma razlike: 9.648645693762933
norma razlike: 9.667824609022922
norma razlike: 9.696013547760671
